In [367]:
from itertools import product

import numpy as np

In [368]:
models = [
    ("cnp", "CNP", "loglik"),
    ("acnp", "ACNP", "loglik"),
    ("convcnp", "ConvCNP", "loglik"),
    ("gnp", "GNP", "loglik"),
    ("agnp", "AGNP", "loglik"),
    ("convgnp", "ConvGNP", "loglik"),
    ("fullconvgnp", "FullConvGNP", "loglik"),
    ("np", "NP", "loglik"),
    ("anp", "ANP", "loglik"),
    ("convnp", "ConvNP", "loglik"),
    ("np", "NP", "elbo"),
    ("anp", "ANP", "elbo"),
    ("convnp", "ConvNP", "elbo"),
]

datasets = [
    "sawtooth",
    # "mixture",
]

In [369]:
def format_line(string, interpolation, extrapolation, out_of_range, phantom=True):
    
    p = "\hphantom{-}" if phantom else ""
    
    if interpolation is not None:
        p1 = p if interpolation[0] > 0 else ""
        inter = f"${p1}{interpolation[0]:8.2f} {{\small \pm {interpolation[1]:6.2f}}}$" if interpolation[0] > -20. else " " * 13 + "F" + " " * 14
    else:
        inter = f"{'Missing':<19}"
        
    if interpolation is not None:
        p2 = p if extrapolation[0] > 0 else ""
        extra = f"${p2}{extrapolation[0]:8.2f} {{\small \pm {extrapolation[1]:6.2f}}}$" if extrapolation[0] > -20. else " " * 13 + "F" + " " * 14
    else:
        extra = f"{'Missing':<19}"
        
    if out_of_range is not None:
        p3 = p if out_of_range[0] > 0 else ""
        oor = f"${p3}{out_of_range[0]:8.2f} {{\small \pm {out_of_range[1]:6.2f}}}$" if out_of_range[0] > -20. else " " * 13 + "F" + " " * 14
    else:
        oor = f"{'Missing':<19}"
    
    return f"{string:<20} & {inter}  & {extra}  & {oor} \\\\"

In [376]:
rows = []
inters = []

datasets = ["sawtooth"]

for (model, string, objective), dataset in product(models, datasets):
    
    root = f"_experiments/{dataset}/x1_y2/{model}"
    
    if "conv" in model:
        root = root + "/unet"
        
    root = root + f"/{objective}/log_evaluate_out.txt"
    
    _file = open(root, "r").read()
    
    try:
        
        if objective == "loglik":

            file = _file.split("| Loglik:")[1].strip().split("\n")[:6]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string, interpolation, extrapolation, out_of_range)
            inters.append(interpolation[0])
            rows.append(row)
            print(row)

            if "cnp" in model and "| AR:" in _file:

                file = _file.split("| AR:")[1].strip().split("\n")[:6]

                process = lambda x: float(x.strip())

                interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
                extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
                out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

                row = format_line(string + " (AR)", interpolation, extrapolation, out_of_range)
                inters.append(interpolation[0])
                rows.append(row)
                print(row)

        else:

            file = _file.split("| Loglik:")[1].strip().split("\n")[:6]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string + ' (EL., ML.)', interpolation, extrapolation, out_of_range)
            inters.append(interpolation[0])
            rows.append(row)
            print(row)

            file = _file.split("| ELBO:")[1].strip().split("\n")[:6]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string + ' (ELBO)', interpolation, extrapolation, out_of_range)
            inters.append(interpolation[0])
            rows.append(row)
            print(row)
            
            
    except:
        row = format_line(string, None, None, None)
        inters.append(-100000)
        print(row)

CNP                  & $   -0.30 {\small \pm   0.00}$  & $   -0.41 {\small \pm   0.00}$  & $   -0.35 {\small \pm   0.00}$ \\
CNP (AR)             & $   -0.29 {\small \pm   0.00}$  & $   -1.19 {\small \pm   0.02}$  & $   -0.89 {\small \pm   0.02}$ \\
ACNP                 & $   -0.08 {\small \pm   0.01}$  & $   -0.41 {\small \pm   0.01}$  & $   -0.42 {\small \pm   0.01}$ \\
ACNP (AR)            & $\hphantom{-}    1.01 {\small \pm   0.01}$  & $   -0.47 {\small \pm   0.01}$  & $   -0.45 {\small \pm   0.01}$ \\
ConvCNP              & $\hphantom{-}    0.87 {\small \pm   0.03}$  & $\hphantom{-}    0.88 {\small \pm   0.02}$  & $   -0.28 {\small \pm   0.00}$ \\
ConvCNP (AR)         & $\hphantom{-}    2.23 {\small \pm   0.01}$  & $\hphantom{-}    2.24 {\small \pm   0.01}$  & $\hphantom{-}    1.94 {\small \pm   0.01}$ \\
GNP                  & $   -0.03 {\small \pm   0.00}$  &              F                &              F               \\
AGNP                 & $\hphantom{-}    0.47 {\small \pm 

In [ ]:
CNP                  & $   -0.17 {\small \pm   0.01}$  & $   -0.18 {\small \pm   0.01}$  & $   -0.18 {\small \pm   0.01}$ \\
CNP (AR)             & $   -0.17 {\small \pm   0.01}$  & $   -0.18 {\small \pm   0.01}$  & $   -0.18 {\small \pm   0.01}$ \\
ACNP                 & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$ \\
ACNP (AR)            & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$ \\
ConvCNP              & $\hphantom{-}    2.40 {\small \pm   0.06}$  & $\hphantom{-}    2.39 {\small \pm   0.05}$  & $\hphantom{-}    0.01 {\small \pm   0.01}$ \\
ConvCNP (AR)         & $\hphantom{-}    3.62 {\small \pm   0.01}$  & $\hphantom{-}    3.62 {\small \pm   0.01}$  & $\hphantom{-}    3.36 {\small \pm   0.02}$ \\
GNP                  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$ \\
AGNP                 & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$ \\
ConvGNP              & $\hphantom{-}    2.87 {\small \pm   0.05}$  & $\hphantom{-}    2.87 {\small \pm   0.05}$  & $   -0.01 {\small \pm   0.01}$ \\
FullConvGNP          & $\hphantom{-}    2.88 {\small \pm   0.07}$  & $\hphantom{-}    2.90 {\small \pm   0.07}$  & $\hphantom{-}    0.30 {\small \pm   0.01}$ \\
NP                   & $\hphantom{-}    0.07 {\small \pm   0.00}$  & $   -0.05 {\small \pm   0.00}$  & $   -0.05 {\small \pm   0.00}$ \\
ANP                  & $\hphantom{-}    0.19 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$  & $   -0.18 {\small \pm   0.00}$ \\
ConvNP               & $\hphantom{-}    3.06 {\small \pm   0.04}$  & $\hphantom{-}    3.06 {\small \pm   0.04}$  & $\hphantom{-}    0.64 {\small \pm   0.01}$ \\
NP (EL., ML.)        & $\hphantom{-}    0.10 {\small \pm   0.01}$  &              F                &              F               \\
NP (ELBO)            & $\hphantom{-}    0.09 {\small \pm   0.01}$  &              F                &              F               \\
ANP (EL., ML.)       & $\hphantom{-}    0.50 {\small \pm   0.02}$  & $   -0.24 {\small \pm   0.00}$  & $   -6.28 {\small \pm   0.46}$ \\
ANP (ELBO)           & $\hphantom{-}    0.50 {\small \pm   0.02}$  & $   -6.28 {\small \pm   0.04}$  & $  -10.81 {\small \pm   0.48}$ \\
ConvNP (EL., ML.)    &              F                &              F                &              F               \\
ConvNP (ELBO)        & $\hphantom{-}    3.51 {\small \pm   0.02}$  & $\hphantom{-}    3.52 {\small \pm   0.02}$  & $\hphantom{-}    2.68 {\small \pm   0.02}$ \\

In [ ]:
CNP                  & $   -0.30 {\small \pm   0.00}$  & $   -0.41 {\small \pm   0.00}$  & $   -0.35 {\small \pm   0.00}$ \\
CNP (AR)             & $   -0.29 {\small \pm   0.00}$  & $   -1.19 {\small \pm   0.02}$  & $   -0.89 {\small \pm   0.02}$ \\
ACNP                 & $   -0.08 {\small \pm   0.01}$  & $   -0.41 {\small \pm   0.01}$  & $   -0.42 {\small \pm   0.01}$ \\
ACNP (AR)            & $\hphantom{-}    1.01 {\small \pm   0.01}$  & $   -0.47 {\small \pm   0.01}$  & $   -0.45 {\small \pm   0.01}$ \\
ConvCNP              & $\hphantom{-}    0.87 {\small \pm   0.03}$  & $\hphantom{-}    0.88 {\small \pm   0.02}$  & $   -0.28 {\small \pm   0.00}$ \\
ConvCNP (AR)         & $\hphantom{-}    2.23 {\small \pm   0.01}$  & $\hphantom{-}    2.24 {\small \pm   0.01}$  & $\hphantom{-}    1.94 {\small \pm   0.01}$ \\
GNP                  & $   -0.03 {\small \pm   0.00}$  &              F                &              F               \\
AGNP                 & $\hphantom{-}    0.47 {\small \pm   0.01}$  & $   -0.38 {\small \pm   0.01}$  & $   -0.70 {\small \pm   0.08}$ \\
ConvGNP              & Missing              & Missing              & Missing             \\
FullConvGNP          & $\hphantom{-}    1.06 {\small \pm   0.03}$  & $\hphantom{-}    1.05 {\small \pm   0.04}$  & $\hphantom{-}    0.11 {\small \pm   0.00}$ \\
NP                   & $   -0.32 {\small \pm   0.00}$  & $   -0.32 {\small \pm   0.00}$  & $   -0.32 {\small \pm   0.00}$ \\
ANP                  & $   -0.21 {\small \pm   0.00}$  & $   -0.33 {\small \pm   0.00}$  & $   -0.33 {\small \pm   0.00}$ \\
ConvNP               & $\hphantom{-}    1.88 {\small \pm   0.03}$  & $\hphantom{-}    1.89 {\small \pm   0.03}$  & $   -0.11 {\small \pm   0.01}$ \\
NP (EL., ML.)        & $   -0.08 {\small \pm   0.01}$  &              F                & $   -1.37 {\small \pm   0.00}$ \\
NP (ELBO)            & $   -0.08 {\small \pm   0.01}$  &              F                & $   -1.37 {\small \pm   0.00}$ \\
ANP (EL., ML.)       & $   -0.04 {\small \pm   0.01}$  & $   -1.09 {\small \pm   0.02}$  & $   -0.35 {\small \pm   0.00}$ \\
ANP (ELBO)           & $   -0.05 {\small \pm   0.01}$  & $   -1.14 {\small \pm   0.03}$  & $   -0.35 {\small \pm   0.00}$ \\
ConvNP (EL., ML.)    & $   -2.98 {\small \pm   0.10}$  & $   -2.98 {\small \pm   0.10}$  & $   -6.74 {\small \pm   0.04}$ \\
ConvNP (ELBO)        & $\hphantom{-}    2.01 {\small \pm   0.01}$  & $\hphantom{-}    2.01 {\small \pm   0.01}$  & $\hphantom{-}    1.47 {\small \pm   0.01}$ \\

In [373]:
for r in np.array(rows)[np.argsort(-np.array(inters))]:
    print(r)

IndexError: index 18 is out of bounds for axis 0 with size 18

In [355]:
len(rows), len(inters)

(19, 19)

In [296]:
rows = []
inters = []
datasets = ["mixture"]

for (model, string, objective), dataset in product(models, datasets):
    
    root = f"_experiments/{dataset}/x1_y1/{model}"
    
    if "conv" in model:
        root = root + "/unet"
        
    root = root + f"/{objective}/log_evaluate_out.txt"
    
    _file = open(root, "r").read()


    try:
        if objective == "loglik":

            file = _file.split("| Loglik:")[1].strip().split("\n")[:12]
            file = file[:2] + file[4:6] + file[8:10]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string, interpolation, extrapolation, out_of_range)
            print(row)

            rows.append(row)
            inters.append(interpolation[0])

            if "cnp" in model and "| AR:" in _file:

                file = _file.split("| AR:")[1].strip().split("\n")[:12]
                file = file[:2] + file[4:6] + file[8:10]

                process = lambda x: float(x.strip())

                interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
                extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
                out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

                row = format_line(string + " (AR)", interpolation, extrapolation, out_of_range)
                print(row)

                rows.append(row)
                inters.append(interpolation[0])

        else:

            file = _file.split("| Loglik:")[1].strip().split("\n")[:12]
            file = file[:2] + file[4:6] + file[8:10]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string + ' (ELBO, ML)', interpolation, extrapolation, out_of_range)

            file = _file.split("| ELBO:")[1].strip().split("\n")[:12]
            file = file[:2] + file[4:6] + file[8:10]

            process = lambda x: float(x.strip())

            interpolation = list(map(process, file[1].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            extrapolation = list(map(process, file[3].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))
            out_of_range = list(map(process, file[5].split("|")[1].strip().split("Loglik (V):")[1].split("(")[0].split("+-")))

            row = format_line(string + ' (ELBO)', interpolation, extrapolation, out_of_range)
            print(row)

            rows.append(row)
            inters.append(interpolation[0])

    except:
        row = format_line(string, None, None, None)
        print(row)

CNP                  &    -1.03 {\small \pm   0.02}  &    -1.10 {\small \pm   0.02}  &    -1.17 {\small \pm   0.02} \\
ACNP                 &    -0.46 {\small \pm   0.02}  &    -1.24 {\small \pm   0.02}  &    -1.22 {\small \pm   0.01} \\
ConvCNP              &     0.07 {\small \pm   0.04}  &     0.06 {\small \pm   0.04}  &    -1.21 {\small \pm   0.01} \\
ConvCNP (AR)         &     0.56 {\small \pm   0.04}  &     0.56 {\small \pm   0.04}  &     0.34 {\small \pm   0.04} \\
GNP                  &    -0.69 {\small \pm   0.01}  &    -2.58 {\small \pm   0.07}  &    -1.74 {\small \pm   0.04} \\
AGNP                 &    -0.34 {\small \pm   0.02}  &    -0.94 {\small \pm   0.02}  &    -1.29 {\small \pm   0.03} \\
ConvGNP              &     0.16 {\small \pm   0.06}  &     0.16 {\small \pm   0.06}  &    -0.91 {\small \pm   0.02} \\
FullConvGNP          &     0.30 {\small \pm   0.03}  &     0.30 {\small \pm   0.03}  &    -0.46 {\small \pm   0.01} \\
NP                   &    -0.63 {\small \pm   0.

In [297]:
for r in np.array(rows)[np.argsort(-np.array(inters))]:
    print(r)

ConvCNP (AR)         &     0.56 {\small \pm   0.04}  &     0.56 {\small \pm   0.04}  &     0.34 {\small \pm   0.04} \\
ConvNP (ELBO)        &     0.45 {\small \pm   0.04}  &     0.45 {\small \pm   0.04}  &    -0.10 {\small \pm   0.04} \\
ConvNP               &     0.38 {\small \pm   0.04}  &     0.39 {\small \pm   0.04}  &    -0.85 {\small \pm   0.02} \\
FullConvGNP          &     0.30 {\small \pm   0.03}  &     0.30 {\small \pm   0.03}  &    -0.46 {\small \pm   0.01} \\
ConvGNP              &     0.16 {\small \pm   0.06}  &     0.16 {\small \pm   0.06}  &    -0.91 {\small \pm   0.02} \\
ConvCNP              &     0.07 {\small \pm   0.04}  &     0.06 {\small \pm   0.04}  &    -1.21 {\small \pm   0.01} \\
ANP                  &    -0.25 {\small \pm   0.02}  &    -1.04 {\small \pm   0.02}  &    -1.04 {\small \pm   0.02} \\
AGNP                 &    -0.34 {\small \pm   0.02}  &    -0.94 {\small \pm   0.02}  &    -1.29 {\small \pm   0.03} \\
ANP (ELBO)           &    -0.40 {\small \pm   0.